# **Personalize seu output Excel com a lib xlsxwriter**

---

**Definição do problema**

Personalizar um arquivo Excel pode demandar bastante tempo, ainda mais quando os dados vêm de fontes externas. Além disso, a padronização desses outputs é fundamental para construção de relatórios de qualidade. Por isso, nada melhor que automatizar essas tarefas!

---

Nesse código, vamos construir um arquivo Excel que mostra um relatório de uma carteira teórica de ativos. Nosso primeiro dataset é composto pelos dados de cotações (abertura, máxima, mínima e fechamento) desses ativos cobrindo um período de um ano. Nosso segundo dataset são os eventos de pagamentos de proventos (ex., dividendos, JCP) dos ativos. O relatório mostrará a participação desses ativos na carteira, valor de mercado inicial e atual e o saldo líquido de cada ativo. Para gerar o output Excel personalizado, utilizaremos a biblioteca xlsxwriter. Busquei cobrir grande parte das seções da documentação para explorar as diferentes funções.

A biblioteca xlsxwriter é uma poderosa ferramenta para a criação de arquivos Excel (.xlsx) em Python. Ela permite uma variedade de formatação de células, inserção de gráficos, fórmulas e funções, além de suporte a imagens e objetos. É compatível com o formato Office Open XML, possui documentação bastante abrangente e uma comunidade bem ativa para consultas.

https://xlsxwriter.readthedocs.io/index.html

https://xlsxwriter.readthedocs.io/workbook.html

https://xlsxwriter.readthedocs.io/format.html

## 1. Bibliotecas

In [1]:
#pip install xlsxwriter

# Manipulação de dados
import numpy as np
import pandas as pd
import datetime
import random

# Formatação output Excel
import xlsxwriter

# Obtenção preços e proventos ativos
import yfinance as yf

## 2. Funções para extração dos dados

**OBS:** Nessa seção vamos construir funções para obter os dados de cotações das ações considerando um período pré-estabelecido. Você pode pular para a próxima seção caso o foco seja apenas na customização do output Excel.

Criamos uma lista com os tickers de seis ativos do IBOV. Os tickers das ações brasileiras do yfinance incluem o ".SA" no final. Para isso, usamos uma compreensão de lista para adicionar .SA em cada item da lista (ticker)

In [2]:
ativos = ['ABEV3', 'ELET3', 'ITUB4', 'PETR4', 'VALE3', 'WEGE3']
ativos_yf = [i + '.SA' for i in ativos]

Definimos as datas de início e fim da nossa análise, usando o formato '%Y-%m-%d' (ex. '2020-01-01'). Vamos definir como padrão a start_date como a data de um ano atrás e o end_date como a data de ontem

In [3]:
start_date=(datetime.datetime.now()-datetime.timedelta(days=366)).strftime('%Y-%m-%d')
end_date=(datetime.datetime.now() - datetime.timedelta(days=1)).strftime('%Y-%m-%d')

Função precos_ativos_yf:

1. Exige os parâmetros ativos_yf, start_date e end_date (todos definidos acima);
2. Cria um dicionário que será preenchido ao longo do loop (mais "correto" que popular um dataframe linha a linha);
3. Para cada ativo (loop), baixa as cotações entre as datas especificadas e armazena a abertura do primeiro dia, máxima das máximas, mínima das mínimas e último preço de fechamento;
4. Ainda no loop, preenche o dicionário nas chaves especificadas;
5. Após o loop, transforma o dicionário em dataframe;
6. Remove o ".SA" dos nomes dos ativos;
7. Formata as colunas de datas para datetime;
8. Retorna o dataframe

In [4]:
def precos_ativos_yf(ativos_yf, start_date=start_date, end_date=end_date):

    dict_precos = {
        "Ativo": [],
        "Data início": [],
        "Data final": [],
        "Abertura": [],
        "Máxima": [],
        "Mínima": [],
        "Fechamento": [],
    }

    for i in ativos_yf:
        df_ohlc = yf.download(
                        i,
                        start=start_date,
                        end=end_date,
                        interval='1d',
                        progress=False
                    )
        abertura = df_ohlc.Open.iloc[0]
        minima = df_ohlc.Low.min()
        maxima = df_ohlc.High.max()
        fechamento = df_ohlc.Close.iloc[-1]

        dict_precos["Ativo"].append(i)
        dict_precos["Data início"].append(start_date)
        dict_precos["Data final"].append(end_date)
        dict_precos["Abertura"].append(abertura)
        dict_precos["Máxima"].append(maxima)
        dict_precos["Mínima"].append(minima)
        dict_precos["Fechamento"].append(fechamento)
    
    df_precos_ativos = pd.DataFrame.from_dict(
        dict_precos, orient='columns')
    
    df_precos_ativos['Ativo'] = df_precos_ativos.Ativo.str.replace('.SA', '')
    df_precos_ativos['Data início'] = pd.to_datetime(df_precos_ativos['Data início'])
    df_precos_ativos['Data final'] = pd.to_datetime(df_precos_ativos['Data final'])
    
    return df_precos_ativos

Função ativos_composicao_carteira:

1. Exige o parâmetros ativos_yf (nossa lista de ativos_yf);
2. Roda a função anterior para gerar o dataframe de preços;
3. Define o random.seed para que os resultados da pseudoaleatorização sejam replicáveis (apenas para que você tenha o mesmo resultado que eu apresento aqui)
4. Cria uma lista composta por números inteiros entre 100 e 1000 e com tamanho igual ao nosso número de ativos;
5. Cria uma coluna com a quantidade comprada de cada ação (não estão em lotes de 100, mas vamos considerar que foram feitas compras também no mercado fracionário)
6. Retorna o dataframe df_composicao_carteira

In [5]:
def ativos_composicao_carteira(ativos_yf):

    df_precos_ativos = precos_ativos_yf(ativos_yf)
    
    ativos = df_precos_ativos.Ativo
    random.seed(3)
    ls_qtd_acoes = random.sample(range(100, 1000), len(ativos))
    df_precos_ativos['Quantidade'] = ls_qtd_acoes

    df_composicao_carteira = df_precos_ativos.copy()
    
    return df_composicao_carteira

Função proventos_ativos_yfinance:

1. Exige os parâmetros ativos_yf, start_date e end_date (definidos acima);
2. Cria um dataframe "df_proventos" vazio;
4. Para cada ativo (loop), extrai os proventos (ex., dividendos, JCP) pagos e suas respectivas datas;
5. Filtra o dataframe de pagamentos considerando apenas o período avaliado;
6. Remove datas onde não houve pagamento para nenhum ativo da lista;
7. Remove o timezone (fuso) da coluna data;
7. Retorna o dataframe df_proventos

In [6]:
def proventos_ativos_yfinance(ativos_yf, start_date=start_date, end_date=end_date):

    df_proventos = pd.DataFrame()

    for i in ativos_yf:
        proventos = yf.Ticker(i).dividends.to_frame(name=i.replace('.SA',''))
        df_proventos = pd.merge(df_proventos, proventos, left_index=True, right_index=True, how='outer')

    df_proventos = df_proventos.loc[start_date:]
    df_proventos = df_proventos.dropna(how='all')

    df_proventos = df_proventos.reset_index()
    df_proventos['Date'] = df_proventos['Date'].dt.tz_localize(None)

    return df_proventos

## 3. Extração dos dados

Vamos gerar os dois dataframes para exportar e customizar nosso relatório

In [7]:
ativos = ['ABEV3', 'ELET3', 'ITUB4', 'PETR4', 'VALE3', 'WEGE3']
ativos_yf = [i + '.SA' for i in ativos]

start_date=(datetime.datetime.now()-datetime.timedelta(days=366)).strftime('%Y-%m-%d')
end_date=(datetime.datetime.now() - datetime.timedelta(days=1)).strftime('%Y-%m-%d')

df_composicao_carteira = ativos_composicao_carteira(ativos_yf)
df_proventos = proventos_ativos_yfinance(ativos_yf)

In [8]:
df_composicao_carteira

,Ativo,Data início,Data final,Abertura,Máxima,Mínima,Fechamento,Quantidade
0,ABEV3,2023-07-10,2024-07-09,15.450000,15.780000,10.980000,11.250000,343
1,ELET3,2023-07-10,2024-07-09,39.520000,44.910000,33.459999,37.880001,706
2,ITUB4,2023-07-10,2024-07-09,29.370001,35.950001,26.360001,32.869999,657
3,PETR4,2023-07-10,2024-07-09,29.110001,42.939999,27.950001,38.439999,233
4,VALE3,2023-07-10,2024-07-09,64.500000,78.550003,59.360001,63.099998,478
5,WEGE3,2023-07-10,2024-07-09,36.959999,44.470001,31.469999,44.470001,718


In [9]:
df_proventos.head()

,Date,ABEV3,ELET3,ITUB4,PETR4,VALE3,WEGE3
0,2023-07-24,NaN,NaN,NaN,NaN,NaN,0.145202
1,2023-08-01,NaN,NaN,0.01765,NaN,NaN,NaN
2,2023-08-14,NaN,NaN,NaN,NaN,1.918472,NaN
3,2023-08-22,NaN,NaN,NaN,1.149304,NaN,NaN
4,2023-09-01,NaN,NaN,0.01765,NaN,NaN,NaN


## 4. Exportação para Excel

Basicamente, a rotina é

* criar o writer;
* preencher as células e definir os formatos/formulas/graficos;
* fechar o writer.

O grande lance está segunda etapa da rotina, onde toda a mágica acontece. Infelizmente não é possível visualizar o que foi feito a cada comando. Só conseguimos visualizar o output depois que fechamos o writer. Uma dica é fragmentar algumas ações desta segunda e executar separadamente (cria writer, fragmenta etapa 2, fecha writer) para visualizar o que aquele comando executa de fato.

Note que é muito mais fácil criar colunas e dados em geral no próprio dataframe e simplesmente exportá-los. Entretanto algumas vezes queremos manter as fórmulas ou formatos específicos do Excel no output. Por isso, mesmo contraintuitivo, vou apresentar algumas delas aqui.

### 4.1. Criar o writer

Primeiro passo será criar um arquivo excel com duas planilhas internas ("Composição carteira" e "Calendário Proventos").

Sugere-se que o formato datetime seja definido inicialmente no parâmetro "datetime_format".

Não é possível formatar células de cabeçalho (header) e índice (index) diretamente. Por isso, observe que para customizar essas linhas/colunas, vamos colocar "header" and "index" como "False" e definir nossa linha inicial ("startrow") para começar em 1.

In [10]:
writer = pd.ExcelWriter('enhanced_excel_output.xlsx', engine='xlsxwriter', datetime_format="dd/mm/yyyy")

df_composicao_carteira.to_excel(writer, sheet_name="Composição_Carteira", startrow=1, header=False, index=False)
df_proventos.to_excel(writer, sheet_name="Calendário_Proventos", startrow=1, header=False, index=False)

workbook = writer.book

### 4.2. Definir estilos de formatação geral

Para maior customização, cada tipo de célula (cabeçalho, índice e demais células) devem receber um estilo específico.

Por exemplo, na formatação do cabeçalho incluiremos funções como tipo/tamanho da fonte, negrito, alinhamento, cor do background e bordas.

Para escolher uma cor basta digitar "#D7E4BC" no Google que abrirá a Tabela de Cores Hexadecimal. Selecione a cor desejada e altere o parâmetro "set_fg_color" do código

In [11]:
header_format = workbook.add_format()
header_format.set_font_name('Arial')
header_format.set_font_size(11)
header_format.set_font_color('black')
header_format.set_bold()
header_format.set_align('center')
header_format.set_align('vcenter')
header_format.set_fg_color("#D7E4BC")
header_format.set_bottom(2)
header_format.set_top(2)

Formato do índice

In [12]:
index_format = workbook.add_format()
index_format.set_font_name('Arial')
index_format.set_font_size(10)
index_format.set_font_color('black')
header_format.set_bold()
index_format.set_align('center')
index_format.set_align('vcenter')

Formato das colunas com preços Abertura, Máxima, Mínima e Fechamento. Queremos que esteja no formato monetário (R$), separador decimal com ".", milhar com "," e duas casas decimais

In [13]:
money_format = workbook.add_format({'num_format': 'R$ #,##0.00'})
money_format.set_font_name('Arial')
money_format.set_font_size(10)
money_format.set_font_color('black')
money_format.set_align('right')
money_format.set_align('vcenter')

Formato coluna Quantidade

In [14]:
integer_format = workbook.add_format()
integer_format.set_font_name('Arial')
integer_format.set_font_size(10)
integer_format.set_font_color('black')
integer_format.set_align('right')
integer_format.set_align('vcenter')

Formato coluna Resultado, em % e com duas casas decimais (essa coluna será criada depois, calculada como o % entre fechamento e abertura (manteremos a fórmula no Excel)

In [15]:
percent_format = workbook.add_format({"num_format": "0.00%"})
percent_format.set_font_name('Arial')
percent_format.set_font_size(10)
percent_format.set_font_color('black')
percent_format.set_align('right')
percent_format.set_align('vcenter')

### 4.3. Formatação subplanilha "Composição_Carteira"

Primeiro definimos a subplanilha "Composição Carteira" que vamos formatar

In [16]:
worksheet_1 = writer.sheets["Composição_Carteira"]

#### 4.3.1. Formatar cabeçalho

Vamos pegar o nome das colunas e iterar para preencher o cabeçalho.

Na função worksheet.write(), definimos a linha inicial, a coluna, o conteúdo (value) da célula e o formato. A linha será 0 para todos (cabeçalho) e o número da coluna sera iterado ao longo do loop

In [17]:
for col_num, value in enumerate(df_composicao_carteira.columns.values):
    worksheet_1.write(0, col_num, value, header_format)

Para entender alguns dos passos, você pode executar o print do que foi feito no loop.
Observe que nesse loop ele pega o índice da coluna (col_num, 0 a 7) e os nomes (value).

In [18]:
for col_num, value in enumerate(df_composicao_carteira.columns.values):
    print(0, col_num, value)

0 0 Ativo
0 1 Data início
0 2 Data final
0 3 Abertura
0 4 Máxima
0 5 Mínima
0 6 Fechamento
0 7 Quantidade


Linha=0, coluna=0, conteúdo=Ativo ....

#### 4.3.2. Formatar índice

Mesmo padrão de loop usado para o cabeçalho, mas agora com "index_num+1" já que a linha 0 representa o cabeçalho. Já o número da coluna será 0 para todos (índice)

In [19]:
for index_num, value in enumerate(df_composicao_carteira.Ativo.values):
    worksheet_1.write(index_num+1, 0, value, index_format)

In [20]:
for index_num, value in enumerate(df_composicao_carteira.Ativo.values):
    print(index_num+1, 0, value)

1 0 ABEV3
2 0 ELET3
3 0 ITUB4
4 0 PETR4
5 0 VALE3
6 0 WEGE3


Linha=1, coluna=0, conteúdo=ABEV3 ....

#### 4.3.3. Formatar colunas específicas: valor monetário

Formatar as colunas Abertura, Máxima, Mínima e Fechamento (D a G no Excel) no formato "money_format" definido anteriormente. Definir o padrão comprimento da coluna como 1 (parâmetro obrigatório mas será alterado com o autofit posteriormente).

In [21]:
worksheet_1.set_column("D:G", 1, money_format)

0

#### 4.3.4. Criar uma nova coluna "Resultado" e manter fórmula

Para mais detalhes sobre fórmulas consulte essa parte da documentação

https://xlsxwriter.readthedocs.io/working_with_formulas.html

Vamos criar a coluna "Resultado" (coluna I), usando o padrão de formatação do cabeçalho

In [22]:
worksheet_1.write('I1', 'Resultado', header_format)

0

Usando um loop, calcular os valores da coluna pela fórmula: ((Fechamento-Abertura)/Abertura *100). Vamos manter a fórmula no output

In [23]:
for index_num, value in enumerate(df_composicao_carteira.Ativo.values):
    celula = "I" + str(index_num+2)
    formula = "=(G" + str(index_num+2) + "-D" + str(index_num + 2) + ")/D"+ str(index_num + 2)
    worksheet_1.write_dynamic_array_formula(celula, formula, percent_format)

Apenas para visualização do loop, vamos printar as variáveis célula e fórmula

In [24]:
for index_num, value in enumerate(df_composicao_carteira.Ativo.values):
    celula = "I" + str(index_num+2)
    formula = "=(G" + str(index_num+2) + "-D" + str(index_num + 2) + ")/D"+ str(index_num + 2)
    worksheet_1.write_dynamic_array_formula(celula, formula, percent_format)
    print(celula)
    print(formula)

I2
=(G2-D2)/D2
I3
=(G3-D3)/D3
I4
=(G4-D4)/D4
I5
=(G5-D5)/D5
I6
=(G6-D6)/D6
I7
=(G7-D7)/D7


Célula I2, função (G2-D2)/D2 ...

#### 4.3.5. Formatação condicional

Formatar os dados da coluna "Resultado" com fundo e cor da fonte verde se o Resultado foi positivo (fechamento > abertura) e vermelho se negativo

In [25]:
format_condition_1 = workbook.add_format({'bg_color': '#FFC7CE',
                               'font_color': '#9C0006'})

format_condition_2 = workbook.add_format({'bg_color':   '#C6EFCE',
                               'font_color': '#006100'})

worksheet_1.conditional_format('I2:I7', {'type': 'cell',
                                        'criteria': '<=',
                                        'value': 0,
                                        'format': format_condition_1})

worksheet_1.conditional_format('I2:I7', {'type': 'cell',
                                        'criteria': '>',
                                        'value': 0,
                                        'format': format_condition_2})

0

#### 4.3.6. Criar duas novas colunas "Valor mercado inicial" e "Valor mercado final" e manter fórmula

Valor mercado inicial = Abertura * quantidade de ações

Valor mercado final = Fechamento * quantidade de ações

In [26]:
worksheet_1.write('J1', 'Valor mercado inicial', header_format)
worksheet_1.write('K1', 'Valor mercado final', header_format)

for index_num, value in enumerate(df_composicao_carteira.Ativo.values):
    celula = "J" + str(index_num+2)
    formula = "=D" + str(index_num+2) + "*H" + str(index_num + 2)
    worksheet_1.write_dynamic_array_formula(celula, formula, money_format)

for index_num, value in enumerate(df_composicao_carteira.Ativo.values):
    celula = "K" + str(index_num+2)
    formula = "=G" + str(index_num+2) + "*H" + str(index_num + 2)
    worksheet_1.write_dynamic_array_formula(celula, formula, money_format)

#### 4.3.7. Adicionar célula abaixo de "Valor mercado inicial" e "Valor mercado final" com o somatório dessas colunas

Para formatar essas células, vamos copiar o estilo das células "money_format" mas incluindo um preenchimento na célula

In [27]:
total_money_format = workbook.add_format({'num_format': 'R$ #,##0.00'})
total_money_format.set_font_name('Arial')
total_money_format.set_font_size(10)
total_money_format.set_font_color('black')
total_money_format.set_bold()
total_money_format.set_align('right')
total_money_format.set_align('vcenter')
total_money_format.set_fg_color("#D7E4BC")

In [28]:
celula = "J" + str(len(df_composicao_carteira.index.values)+2)
formula = "=sum(J2:J" + str(len(df_composicao_carteira.index.values) + 1) + ")"
worksheet_1.write_dynamic_array_formula(celula, formula, cell_format=total_money_format)

celula = "K" + str(len(df_composicao_carteira.index.values)+2)
formula = "=sum(K2:K" + str(len(df_composicao_carteira.index.values) + 1) + ")"
worksheet_1.write_dynamic_array_formula(celula, formula, cell_format=total_money_format)

0

#### 4.3.8. Adicionar coluna com Saldo da variação do preço

Saldo da variação do preço = Valor mercado final - Valor mercado inicial

In [29]:
worksheet_1.write('L1', 'Saldo variação preço', header_format)

for index_num, value in enumerate(df_composicao_carteira.Ativo.values):
    celula = "L" + str(index_num+2)
    formula = "=K" + str(index_num+2) + "-J" + str(index_num + 2)
    worksheet_1.write_dynamic_array_formula(celula, formula, money_format)

celula = "L" + str(len(df_composicao_carteira.index.values)+2)
formula = "=sum(L2:L" + str(len(df_composicao_carteira.index.values) + 1) + ")"
worksheet_1.write_dynamic_array_formula(celula, formula, cell_format=total_money_format)

0

### 4.4. Formatação subplanilha "Calendário_Proventos"

Seguiremos o mesmo código da outra subplanilha

In [30]:
worksheet_2 = writer.sheets["Calendário_Proventos"]

### 4.4.1. Cabeçalho

In [31]:
for col_num, value in enumerate(df_proventos.columns.values):
    worksheet_2.write(0, col_num, value, header_format)

#### 4.4.2. Criar uma última linha "TOTAL"

Incluiremos uma linha no final da subplanilha, com o total de proventos pagos por cada ativo no período considerado. Esse total representa quanto uma única ação teria gerado de proventos

Dessa vez, a fórmula abrange uma mesma linha e diferentes colunas (mais complexo). Anteriormente vimos uma fórmula aplicada a diferentes linhas da mesma coluna

Para manter a fórmula, faremos um pouco de malabarismo nos dados usando a lib string. Ela exporta a sequencia de letras (ABCD...), que usaremos no loop para construir a fórmula que soma todas as linhas acima dentro de uma mesma coluna

Nossa coluna A são as datas de pagamentos, entao de B a G temos nossos ativos (índice 1 a número de colunas do df)

In [32]:
import string

uppercase_alphabet = string.ascii_uppercase
str_columns = uppercase_alphabet[1:len(df_proventos.columns)]
str_columns

'BCDEFG'

In [33]:
worksheet_2.write(str("A"+str(len(df_proventos)+2)), 'TOTAL', header_format)

for i in str_columns:
    celula = str(i + str(len(df_proventos)+2))
    formula = "=sum("+i+'2:'+str(i + str(len(df_proventos)+1))+")"
    worksheet_2.write_dynamic_array_formula(celula, formula, cell_format=total_money_format)
    print(celula)
    print(formula)

B33
=sum(B2:B32)
C33
=sum(C2:C32)
D33
=sum(D2:D32)
E33
=sum(E2:E32)
F33
=sum(F2:F32)
G33
=sum(G2:G32)


### 4.5. Acessar dados de outra subplanilha

Criar uma nova coluna "Total proventos recebidos" na subplanilha "Composição_Carteira"

Essa coluna será a multiplicação do total pago por ação no intervalo de tempo avaliado (Calendário Proventos) e a quantidade de cada ação na carteira

In [34]:
worksheet_1.write("M1", 'Total proventos recebidos', header_format)

for index_num, value in enumerate(df_composicao_carteira.Ativo.values):
    celula = "M" + str(index_num+2)
    formula = "=Calendário_Proventos!" + str_columns[index_num]+'33' + "*H" + str(index_num + 2)
    worksheet_1.write_dynamic_array_formula(celula, formula, money_format)

Adicionar uma célula abaixo com o total de proventos recebidos

In [35]:
celula = "M" + str(len(df_composicao_carteira.index.values)+2)
formula = "=sum(M2:M" + str(len(df_composicao_carteira.index.values) + 1) + ")"
worksheet_1.write_dynamic_array_formula(celula, formula, cell_format=total_money_format)

0

### 4.6. Inserir gráficos

Seções da documentação

https://xlsxwriter.readthedocs.io/chart.html#chart-class

https://xlsxwriter.readthedocs.io/working_with_charts.html#working-with-charts

Primeiro vamos inserir um gráfico de pizza com a composição de nossa carteira por ativos

In [36]:
chart_pizza = workbook.add_chart({'type': 'pie'})

chart_pizza.add_series({
    "name": "Distribuição carteira",
    'categories': '=Composição_Carteira!$A$2:$A$7',
    'values': '=Composição_Carteira!$H$2:$H$7',
    'border': {'color': 'black'},
    "data_labels": {"value": True}
})

chart_pizza.set_title({"name": "Composição da carteira"})

worksheet_1.insert_chart('H11', chart_pizza)

0

Agora um gráfico de barras empilhadas para avaliar o saldo da carteira resultante da variação do preço do ativo e o total de proventos pagos.

Nesse caso, é preciso configurar as duas séries de dados

In [37]:
chart_bar = workbook.add_chart({"type": "bar", "subtype": "stacked"})

chart_bar.add_series(
    {
        "name": "=Composição_Carteira!$L$1",
        "categories": "=Composição_Carteira!$A$2:$A$7",
        "values": "=Composição_Carteira!$L$2:$L$7",
    }
)

chart_bar.add_series(
    {
        "name": "=Composição_Carteira!$M$1",
        "categories": "=Composição_Carteira!$A$2:$A$7",
        "values": "=Composição_Carteira!$M$2:$M$7",
    }
)

chart_bar.set_title({"name": "Saldo líquido ativos"})
chart_bar.set_x_axis({"name": "Valor (R$)",
                      'crossing': 'min'})
chart_bar.set_y_axis({"name": "Ativos",
                      'crossing': 'min'})
chart_bar.set_legend({'position': 'bottom'})

worksheet_1.insert_chart("B11", chart_bar, {"x_offset": 25, "y_offset": 10})

0

### 4.7. Inserir uma imagem

Vamos baixar a imagem da lib yfinance pelo link e inserir na célula O2. Reduzir o tamanho pela metade

In [38]:
from PIL import Image
import requests
from io import BytesIO

url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/8/8f/Yahoo%21_Finance_logo_2021.png/375px-Yahoo%21_Finance_logo_2021.png'
response = requests.get(url)
img = Image.open(BytesIO(response.content))
img.save("yfinance_fig.png","PNG")

worksheet_1.insert_image('O2', "yfinance_fig.png", {'x_scale': 0.5, 'y_scale': 0.5})

0

### 4.8. Fechar o writer

Por fim, configuramos o ajuste automático do tamanho das colunas em cada subplanilha e fechamos o writer 

In [39]:
worksheet_1.autofit()
worksheet_2.autofit()

writer.close()

Pronto, agora só checar como ficou seu arquivo Excel na pasta e explorar o universo de funções, formatos e gráficos disponíveis!